In [1]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

import xarray as xr
import dask
xr.set_options(display_style="html")
import pandas as pd
import numpy as np
from numba import jit
from functools import reduce

import glob
import os

import time
import tqdm.notebook as tq

import warnings
warnings.filterwarnings("ignore")

In [2]:
def select_big_from_MP(WS_geometry):
    """
    
    Function return only biggest polygon 
    from multipolygon WS
    It's the real WS, and not malfunctioned part of it
    
    """
    if type(WS_geometry) == MultiPolygon:
        big_area = [polygon_area(lats=polygon.exterior.coords.xy[1],
                                 lons=polygon.exterior.coords.xy[0])
                    for polygon in WS_geometry]
        WS_geometry = WS_geometry[np.argmax(big_area)]
    else:
        WS_geometry = WS_geometry
    return WS_geometry


def polygon_area(lats, lons, radius=6378137):
    """
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified.
    Otherwise, in the units of provided radius.
    lats and lons are in degrees.
    """
    from numpy import (arctan2, cos, sin, sqrt,
                       pi, append, diff)

    lats, lons = np.deg2rad(lats), np.deg2rad(lons)
    # Line integral based on Green's Theorem, assumes spherical Earth

    #close polygon
    if lats[0] != lats[-1]:
        lats = append(lats, lats[0])
        lons = append(lons, lons[0])

    #colatitudes relative to (0,0)
    a = sin(lats/2)**2 + cos(lats) * sin(lons/2)**2
    colat = 2*arctan2(sqrt(a), sqrt(1-a))

    #azimuths relative to (0,0)
    az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

    # Calculate diffs
    # daz = diff(az) % (2*pi)
    daz = diff(az)
    daz = (daz + pi) % (2 * pi) - pi

    deltas = diff(colat)/2
    colat = colat[0:-1]+deltas

    # Perform integral
    integrands = (1-cos(colat)) * daz

    # Integrate
    area = abs(sum(integrands))/(4*pi)

    area = min(area, 1-area)
    if radius is not None:  # return in units of radius
        return area * 4 * pi * radius**2 / 10**6
    else:  # return in ratio of sphere total area
        return area / 10**6


def find_extent(ws):
    """
    ws - geometry of watershed of interest

    Return list of exterior of watershed by 0.25 degree
    (equal to ERA5 grid)
    """

    def x_round(x):
        return round(x*4)/4

    LONS, LATS = ws.exterior.xy
    max_LAT = max(LATS)
    max_LON = max(LONS)
    min_LAT = min(LATS)
    min_LON = min(LONS)

    return [x_round(min_LON), x_round(max_LON),
            x_round(min_LAT), x_round(max_LAT)]


def create_GDF(shape):
    """
    
    create geodataframe with given shape
    as a geometry
    
    """
    gdf_your_WS = select_big_from_MP(WS_geometry=shape)
    ### WS from your data
    gdf_your_WS = gpd.GeoSeries([gdf_your_WS])

    ### Create extra gdf to use geopandas functions
    gdf_your_WS = gpd.GeoDataFrame({'geometry': gdf_your_WS})
    gdf_your_WS = gdf_your_WS.set_crs('EPSG:4326')

    return gdf_your_WS


def select_NC_by_extent(nc, shape):
    """
    
    select net_cdf by extent of given shape
    
    return masked net_cdf
    
    """
    if 'latitude' in nc.dims:
        nc = nc.rename({'latitude': 'lat', 'longitude': 'lon'})
    else:
        pass

    # find biggest polygon
    big_shape = select_big_from_MP(WS_geometry=shape)

    # find extent coordinates
    min_LON, max_LON, min_LAT, max_LAT = find_extent(ws=big_shape)

    # select nc inside of extent
    masked_nc = nc.where(
        nc.lat >= min_LAT, drop=True).where(
        nc.lat <= max_LAT, drop=True).where(
        nc.lon >= min_LON, drop=True).where(
        nc.lon <= max_LON, drop=True)
    masked_nc = masked_nc.chunk(chunks='auto')
    return masked_nc


def RotM(alpha):
    """ Rotation Matrix for angle ``alpha`` """
    sa, ca = np.sin(alpha), np.cos(alpha)
    return np.array([[ca, -sa],
                     [sa,  ca]])


def getSquareVertices(mm, h, phi):
    """ Calculate the for vertices for square with center ``mm``,
        side length ``h`` and rotation ``phi`` """
    hh0 = np.ones(2)*h  # initial corner
    vv = [np.asarray(mm) + reduce(np.dot, [RotM(phi), RotM(np.pi/2*c), hh0])
          for c in range(4)]  # rotate initial corner four times by 90°
    return np.asarray(vv)


def ERA_5_time_series_for_WS(ws_ID_row, geometry_row, ERA5_nc, path_to_save):

    variables = list(ERA5_nc.data_vars)

    if len(variables) != 1:
        print('Something went wrong {}'.format(variables))
    else:
        start_time = time.time()
        print('\nVariable {} starts calculation'.format(variables[0]))
        # redefine path_to_save
        # path_to_save = path_to_save + variables[0]

        os.makedirs(path_to_save, exist_ok=True)

        for ii, test_shape in enumerate(tq.tqdm(geometry_row)):
            # transform selected shape to geodataframe
            # for further manipulations
            big_shape = create_GDF(test_shape)
            # use mask on net_cdf
            with dask.config.set(**{'array.slicing.split_large_chunks': True}):
                mask_nc = select_NC_by_extent(nc=ERA5_nc[variables[0]],
                                              shape=test_shape)

            # get lat, lon which help define area for intersection
            nc_lat, nc_lon = mask_nc.lat.values, mask_nc.lon.values

            # emulate polygons for ERA5 grid
            polygons = list()
            for j in range(nc_lat.size):
                for i in range(nc_lon.size):
                    # h = 0.125 as a half of ERA5 resolution
                    # phi rotation angle
                    polygons.append(Polygon(getSquareVertices(mm=(nc_lon[i], nc_lat[j]),
                                                              h=0.125,
                                                              phi=0)))
            # create geodataframe from each polygon from emulation
            polygons = [create_GDF(poly) for poly in polygons]
            # find intersection beetween grid cell and actual watershed

            intersected = list()
            for polygon in polygons:
                try:
                    intersected.append(gpd.overlay(df1=big_shape,
                                                   df2=polygon,
                                                   how='intersection'))
                except KeyError:
                    intersected.append(gpd.GeoDataFrame())
            # find biggest intersection if it returns MultiPolygon instance
            # convert to Polygon
            intersected = [select_big_from_MP(section['geometry'][0])
                           if len(section) != 0
                           else gpd.GeoDataFrame()
                           for section in intersected]

            # create mask for intersection with net_cdf
            inter_mask = np.array([section.empty != True
                                   for section in intersected])
            # [1:3] is used because first dimension is time
            inter_mask = inter_mask.reshape(mask_nc.shape[1:3])
            inter_mask = xr.DataArray(data=inter_mask,
                                      dims=['lat', 'lon'],
                                      coords=[nc_lat, nc_lon])
            # create final instersection
            WS_nc = mask_nc.where(inter_mask, drop=True)

            # calculate weights of each intersection correspond to ERA grid cell
            weights = np.array([0 if isinstance(section, gpd.GeoDataFrame)
                                else polygon_area(lats=section.exterior.xy[1],
                                                  lons=section.exterior.xy[0]) /
                                polygon_area(lats=polygons[i]['geometry'][0].exterior.xy[1],
                                             lons=polygons[i]['geometry'][0].exterior.xy[0])
                                for i, section in enumerate(intersected)])
            # transform to form on NetCDF
            weights = weights.reshape(WS_nc.shape[1:3])
            # transform to DataArray for calculations
            weights = xr.DataArray(data=weights,
                                   dims=['lat', 'lon'])
            weights.name = 'weights'

            if 't2m' in variables:
                # record DataFrame
                result_df = pd.DataFrame()
                result_df['date'] = WS_nc.time.values
                result_df[variables[0]] = (
                    (WS_nc * weights).sum(dim=['lat', 'lon']) / np.sum(weights)).values
                if os.path.isfile('{}/{}.csv'.format(path_to_save,
                                                     ws_ID_row.loc[ii])):
                    pass
                else:
                    result_df.to_csv('{}/{}.csv'.format(path_to_save,
                                                        ws_ID_row.loc[ii]),
                                     index=False)
                end_time = time.time()
            else:
                # record DataFrame
                result_df = pd.DataFrame()
                result_df['date'] = WS_nc.time.values
                result_df[variables[0]] = (
                    (WS_nc * weights).sum(dim=['lat', 'lon']))
                if os.path.isfile('{}/{}.csv'.format(path_to_save,
                                                     ws_ID_row.loc[ii])):
                    pass
                else:
                    result_df.to_csv('{}/{}.csv'.format(path_to_save,
                                                        ws_ID_row.loc[ii]),
                                     index=False)
                end_time = time.time()
        print("Variable {} successfully calculated in {:.2f} minutes".format(variables[0],
                                                                             (end_time - start_time) / 60))

        return


In [3]:
conus_basin_path = "S:/education/HSI/aspirantura/Dissertation/conus_data/basin_set_full_res/HCDN_nhru_final_671.shp"
CONUS_WS = gpd.read_file(conus_basin_path)
# select only biggest polygon from mulitipolygons file
# if any exists
CONUS_WS['geometry'] = [select_big_from_MP(ws) 
                         for ws in CONUS_WS['geometry']]

In [5]:
# get variables for computation
var_names = [i.split('\\')[-1] 
             for i in 
             glob.glob('E:/ERA5_data/CONUS/daily/*')]
# define paths
data_paths = ('E:/ERA5_data/CONUS/daily/2m_temperature/*.nc',
              'E:/ERA5_data/CONUS/daily/2m_temperature_max/*.nc',
              'E:/ERA5_data/CONUS/daily/2m_temperature_min/*.nc',
              'E:/ERA5_data/CONUS/daily/evaporation/*.nc',
              'E:/ERA5_data/CONUS/daily/potential_evaporation/*.nc',
              'E:/ERA5_data/CONUS/daily/runoff/*.nc',
              'E:/ERA5_data/CONUS/daily/total_precipitation/*.nc',
              'E:/ERA5_data/CONUS/daily/mean_surface_direct_short_wave_radiation_flux/*.nc',
              'E:/ERA5_data/CONUS/daily/mean_surface_direct_short_wave_radiation_flux/*.nc',
              'E:/ERA5_data/CONUS/daily/mean_surface_direct_short_wave_radiation_flux_max/*.nc',
              'E:/ERA5_data/CONUS/daily/mean_surface_direct_short_wave_radiation_flux_min/*.nc')


In [6]:
for i, path in enumerate(data_paths):
    print('\nCalculation for {} files'.format(var_names[i]))
    nc_data = xr.open_mfdataset(paths=path,
                                engine='netcdf4',
                                chunks='auto')
    ERA_5_time_series_for_WS(ws_ID_row=CONUS_WS['hru_id'],
                             geometry_row=CONUS_WS['geometry'],
                             ERA5_nc=nc_data,
                             path_to_save='S:/education/HSI/aspirantura/Dissertation/conus_data/WS_by_VARIABLE/{data_fodler}'.format(data_fodler=path.split('/')[-2]))



Calculation for mean_surface_direct_short_wave_radiation_flux files

Variable msdrswrf starts calculation


  0%|          | 0/671 [00:00<?, ?it/s]

Variable msdrswrf successfully calculated in 19.71 minutes

Calculation for mean_surface_direct_short_wave_radiation_flux_max files

Variable msdrswrf starts calculation


  0%|          | 0/671 [00:00<?, ?it/s]

Variable msdrswrf successfully calculated in 457.71 minutes

Calculation for mean_surface_direct_short_wave_radiation_flux_min files

Variable msdrswrf starts calculation


  0%|          | 0/671 [00:00<?, ?it/s]

Variable msdrswrf successfully calculated in 15.97 minutes
